# Tests Pyfhel

In [1]:
import numpy as np
from Pyfhel import Pyfhel

In [2]:
HE = Pyfhel()           # Creating empty Pyfhel object
bfv_params = {
    'scheme': 'BFV',    # can also be 'bfv'
    'n': 2**13,         # Polynomial modulus degree, the num. of slots per plaintext,
                        #  of elements to be encoded in a single ciphertext in a
                        #  2 by n/2 rectangular matrix (mind this shape for rotations!)
                        #  Typ. 2^D for D in [10, 16]
    't': 65537,         # Plaintext modulus. Encrypted operations happen modulo t
                        #  Must be prime such that t-1 be divisible by 2^N.
    't_bits': 20,       # Number of bits in t. Used to generate a suitable value 
                        #  for t. Overrides t if specified.
    'sec': 128,         # Security parameter. The equivalent length of AES key in bits.
                        #  Sets the ciphertext modulus q, can be one of {128, 192, 256}
                        #  More means more security but also slower computation.
}
HE.contextGen(**bfv_params)  # Generate context for bfv scheme
HE.keyGen()             # Key Generation: generates a pair of public/secret keys
HE.rotateKeyGen()       # Rotate key generation --> Allows rotation/shifting
HE.relinKeyGen()        # Relinearization key generation

In [3]:
arr1 = np.array([1.0, 2.0, 3.0], dtype=np.double)    # Max possible value is t/2-1.
arr2 = np.array([2.0, 3.0, 4.0], dtype=np.double)    # Min possible value is -t/2. 

ptxt1 = HE.encode(arr1)   # Creates a PyPtxt plaintext with the encoded arr1
ptxt2 = HE.encode(arr2)   # plaintexts created from arrays shorter than 'n' are filled with zeros.

ctxt1 = HE.encryptPtxt(ptxt1) # Encrypts the plaintext ptxt1 and returns a PyCtxt
ctxt2 = HE.encryptPtxt(ptxt2) #  Alternatively you can use HE.encryptInt(arr2)

# Otherwise, a single call to `HE.encrypt` would detect the data type,
#  encode it and encrypt it
#> ctxt1 = HE.encrypt(arr1)

In [4]:
# Ciphertext-ciphertext ops:
ccSum = ctxt1 + ctxt2       # Calls HE.add(ctxt1, ctxt2, in_new_ctxt=True)
                            #  `ctxt1 += ctxt2` for inplace operation
ccSub = ctxt1 - ctxt2       # Calls HE.sub(ctxt1, ctxt2, in_new_ctxt=True)
                            #  `ctxt1 -= ctxt2` for inplace operation
ccMul = ctxt1 * ctxt2       # Calls HE.multiply(ctxt1, ctxt2, in_new_ctxt=True)
                            #  `ctxt1 *= ctxt2` for inplace operation
~ccMul                      # relinearize
    
# Single Ciphertext ops:
cSq   = ctxt1**2            # Calls HE.square(ctxt1, in_new_ctxt=True)
                            #  `ctxt1 **= 2` for inplace operation
cNeg  = -ctxt1              # Calls HE.negate(ctxt1, in_new_ctxt=True)
                            # 
cPow  = ctxt1**3            # Calls HE.power(ctxt1, 3, in_new_ctxt=True)
                            #  `ctxt1 **= 3` for inplace operation
cRotR = ctxt1 >> 2          # Calls HE.rotate(ctxt1, k=2, in_new_ctxt=True)
                            #  `ctxt1 >>= 2` for inplace operation
                            # WARNING! the encoded data is placed in a n//2 by 2
                            #  matrix. Hence, these rotations apply independently
                            #  to each of the rows!
cRotL = ctxt1 << 2          # Calls HE.rotate(ctxt1, k=-2, in_new_ctxt=True)
                            #  `ctxt1 <<= 2` for inplace operation

# Ciphertext-plaintext ops
cpSum = ctxt1 + ptxt2       # Calls HE.add_plain(ctxt1, ptxt2, in_new_ctxt=True)
                            # `ctxt1 += ctxt2` for inplace operation
cpSub = ctxt1 - ptxt2       # Calls HE.sub_plain(ctxt1, ptxt2, in_new_ctxt=True)
                            # `ctxt1 -= ctxt2` for inplace operation
cpMul = ctxt1 * ptxt2       # Calls HE.multiply_plain(ctxt1, ptxt2, in_new_ctxt=True)
                            # `ctxt1 *= ctxt2` for inplace operation

In [5]:
r1     = HE.decrypt(ctxt1)
r2     = HE.decrypt(ctxt2)
rccSum = HE.decrypt(ccSum)
rccSub = HE.decrypt(ccSub)
rccMul = HE.decrypt(ccMul)
rcSq   = HE.decrypt(cSq  )
rcNeg  = HE.decrypt(cNeg )
rcPow  = HE.decrypt(cPow )
rcRotR = HE.decrypt(cRotR)
rcRotL = HE.decrypt(cRotL)
rcpSum = HE.decrypt(cpSum)
rcpSub = HE.decrypt(cpSub)
rcpMul = HE.decrypt(cpMul)

print(" Original ciphertexts: ")
print("   ->\tctxt1 --(decr)--> ", r1)
print("   ->\tctxt2 --(decr)--> ", r2)
print(" Ciphertext-ciphertext Ops: ")
print("   ->\tctxt1 + ctxt2 = ccSum --(decr)--> ", rccSum)
print("   ->\tctxt1 - ctxt2 = ccSub --(decr)--> ", rccSub)
print("   ->\tctxt1 * ctxt2 = ccMul --(decr)--> ", rccMul)
print(" Single ciphertext: ")
print("   ->\tctxt1**2      = cSq   --(decr)--> ", rcSq  )
print("   ->\t- ctxt1       = cNeg  --(decr)--> ", rcNeg )
print("   ->\tctxt1**3      = cPow  --(decr)--> ", rcPow )
print("   ->\tctxt1 >> 2    = cRotR --(decr)--> ", rcRotR)
print("   ->\tctxt1 << 2    = cRotL --(decr)--> ", rcRotL)
print(" Ciphertext-plaintext ops: ")
print("   ->\tctxt1 + ptxt2 = cpSum --(decr)--> ", rcpSum)
print("   ->\tctxt1 - ptxt2 = cpSub --(decr)--> ", rcpSub)
print("   ->\tctxt1 * ptxt2 = cpMul --(decr)--> ", rcpMul)

 Original ciphertexts: 
   ->	ctxt1 --(decr)-->  [1 2 3 ... 0 0 0]
   ->	ctxt2 --(decr)-->  [2 3 4 ... 0 0 0]
 Ciphertext-ciphertext Ops: 
   ->	ctxt1 + ctxt2 = ccSum --(decr)-->  [3 5 7 ... 0 0 0]
   ->	ctxt1 - ctxt2 = ccSub --(decr)-->  [-1 -1 -1 ...  0  0  0]
   ->	ctxt1 * ctxt2 = ccMul --(decr)-->  [ 2  6 12 ...  0  0  0]
 Single ciphertext: 
   ->	ctxt1**2      = cSq   --(decr)-->  [1 4 9 ... 0 0 0]
   ->	- ctxt1       = cNeg  --(decr)-->  [-1 -2 -3 ...  0  0  0]
   ->	ctxt1**3      = cPow  --(decr)-->  [ 1  8 27 ...  0  0  0]
   ->	ctxt1 >> 2    = cRotR --(decr)-->  [0 0 1 ... 0 0 0]
   ->	ctxt1 << 2    = cRotL --(decr)-->  [3 0 0 ... 0 0 0]
 Ciphertext-plaintext ops: 
   ->	ctxt1 + ptxt2 = cpSum --(decr)-->  [3 5 7 ... 0 0 0]
   ->	ctxt1 - ptxt2 = cpSub --(decr)-->  [-1 -1 -1 ...  0  0  0]
   ->	ctxt1 * ptxt2 = cpMul --(decr)-->  [ 2  6 12 ...  0  0  0]
